#### Get started with Network Sciecne Simulation

Import all required libs

In [9]:
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import random
import ipywidgets as widgets
from IPython.display import display
import seaborn as sns

### Simulation Parameters:
- **NUM_AGENTS**: Number of agents in the network. This controls the size of the simulation.
- **INITIAL_COOPERATION_PROB**: The probability that an agent starts as a cooperator.
- **INEQUALITY_LEVEL**: Controls the inequality of resource distribution using a Pareto distribution. Higher values mean more inequality.
- **NUM_TIME_STEPS**: The number of time steps over which the simulation runs.


In [10]:

NUM_AGENTS = 100
INITIAL_COOPERATION_PROB = 0.5
INEQUALITY_LEVEL = 2.0
NUM_TIME_STEPS = 50

Parameters of Agents

In [11]:
# Payoff Matrix (T > R > P > S)
# T: Temptation to defect
# R: Reward for mutual cooperation
# P: Punishment for mutual defection
# S: Sucker's payoff
T = 5
R = 3
P = 2
S = 0

Define function for simulation

In [12]:



def assign_resources(G, inequality_level):
    # Use Pareto distribution to assign resources (wealth)
    # Higher inequality_level means more inequality
    m = 1  # Scale parameter
    resources = (np.random.pareto(inequality_level, NUM_AGENTS) + 1) * m
    total_resources = np.sum(resources)
    normalized_resources = resources / total_resources  # Normalize to sum to 1
    for i, node in enumerate(G.nodes()):
        G.nodes[node]['resource'] = normalized_resources[i]

def initialize_strategies(G, cooperation_prob):
    for node in G.nodes():
        if random.random() < cooperation_prob:
            G.nodes[node]['strategy'] = 'C'
        else:
            G.nodes[node]['strategy'] = 'D'

def play_game(G):
    payoffs = {}
    for node in G.nodes():
        strategy = G.nodes[node]['strategy']
        resource = G.nodes[node]['resource']
        payoff = 0
        for neighbor in G.neighbors(node):
            neighbor_strategy = G.nodes[neighbor]['strategy']
            # Adjust payoffs based on own resource level (optional complexity)
            if strategy == 'C' and neighbor_strategy == 'C':
                payoff += R
            elif strategy == 'C' and neighbor_strategy == 'D':
                payoff += S
            elif strategy == 'D' and neighbor_strategy == 'C':
                payoff += T
            elif strategy == 'D' and neighbor_strategy == 'D':
                payoff += P
        # Total payoff adjusted by own resource level
        payoffs[node] = payoff * resource
    return payoffs

def update_strategies(G, payoffs):
    new_strategies = {}
    for node in G.nodes():
        # Select a random neighbor
        neighbors = list(G.neighbors(node))
        if neighbors:
            neighbor = random.choice(neighbors)
            # Compare payoffs
            if payoffs[neighbor] > payoffs[node]:
                # Adopt neighbor's strategy with probability proportional to payoff difference
                prob = (payoffs[neighbor] - payoffs[node]) / (max(payoffs.values()) - min(payoffs.values()) + 1e-6)
                if random.random() < prob:
                    new_strategies[node] = G.nodes[neighbor]['strategy']
                else:
                    new_strategies[node] = G.nodes[node]['strategy']
            else:
                new_strategies[node] = G.nodes[node]['strategy']
        else:
            new_strategies[node] = G.nodes[node]['strategy']
    # Update strategies
    for node in G.nodes():
        G.nodes[node]['strategy'] = new_strategies[node]

def gini_coefficient(resources):
    sorted_resources = sorted(resources)
    n = len(resources)
    cumulative = np.cumsum(sorted_resources)
    gini_index = (2 * np.sum(cumulative)) / (n * np.sum(sorted_resources)) - (n + 1) / n
    return gini_index

Simulate

In [13]:
def simulate():
    G = nx.erdos_renyi_graph(NUM_AGENTS, 0.1)  # Example of a random network
    
    assign_resources(G, INEQUALITY_LEVEL)
    initialize_strategies(G, INITIAL_COOPERATION_PROB)
    
    cooperation_levels = []
    resource_distribution = [G.nodes[node]['resource'] for node in G.nodes()]

    for t in range(NUM_TIME_STEPS):
        payoffs = play_game(G)
        update_strategies(G, payoffs)
        
        # Measure cooperation level
        num_cooperators = sum(1 for node in G.nodes() if G.nodes[node]['strategy'] == 'C')
        cooperation_level = num_cooperators / NUM_AGENTS
        cooperation_levels.append(cooperation_level)
        #print(f"Time Step {t+1}: Cooperation Level = {cooperation_level:.2f}")

    return cooperation_levels, resource_distribution, G


In [2]:
import matplotlib.pyplot as plt
import numpy as np
import random
import networkx as nx

# Adjust grid size to 100x100
GRID_SIZE = 100  # 100x100 lattice
NUM_AGENTS = GRID_SIZE * GRID_SIZE  # Total number of agents = 100^2 = 10,000
RICH_SITE_PROB = 0.2  # Fraction of rich sites
INITIAL_COOPERATION_PROB = 0.5
INEQUALITY_LEVEL = 10  # Resource contrast (c) between rich and poor sites
TEMPTATION_LEVEL = 0.7  # High temptation (b-1)

# Payoff matrix parameters
RICH_RESOURCE = 1 + INEQUALITY_LEVEL
POOR_RESOURCE = 1
b = 2 / TEMPTATION_LEVEL  # Public goods multiplier

# Create a 2D grid of agents (lattice)
def create_lattice():
    G = nx.grid_2d_graph(GRID_SIZE, GRID_SIZE, periodic=True)  # 2D grid with periodic boundary
    assign_resources(G)
    initialize_strategies(G, INITIAL_COOPERATION_PROB)
    return G

# Assign resources to rich and poor sites
def assign_resources(G):
    for node in G.nodes():
        if random.random() < RICH_SITE_PROB:
            G.nodes[node]['resource'] = RICH_RESOURCE
            G.nodes[node]['site_type'] = 'rich'
        else:
            G.nodes[node]['resource'] = POOR_RESOURCE
            G.nodes[node]['site_type'] = 'poor'

# Initialize strategies for agents (either cooperate 'C' or defect 'D')
def initialize_strategies(G, cooperation_prob):
    for node in G.nodes():
        G.nodes[node]['strategy'] = 'C' if random.random() < cooperation_prob else 'D'

# Play the game for one time step and calculate payoffs
def play_game(G):
    payoffs = {}
    for node in G.nodes():
        strategy = G.nodes[node]['strategy']
        resource = G.nodes[node]['resource']
        payoff = 0
        neighbors = G.neighbors(node)
        
        for neighbor in neighbors:
            neighbor_strategy = G.nodes[neighbor]['strategy']
            neighbor_resource = G.nodes[neighbor]['resource']

            # Payoff matrix from the paper (rich and poor site interactions)
            if strategy == 'C' and neighbor_strategy == 'C':
                payoff += neighbor_resource  # Cooperation between cooperators
            elif strategy == 'C' and neighbor_strategy == 'D':
                payoff += 0  # Cooperator interacting with defector
            elif strategy == 'D' and neighbor_strategy == 'C':
                payoff += b * resource  # Defector interacting with cooperator
            elif strategy == 'D' and neighbor_strategy == 'D':
                payoff += 0  # Defector interacting with defector

        payoffs[node] = payoff
    return payoffs

# Update strategies based on payoffs and stochastic imitation
def update_strategies(G, payoffs):
    new_strategies = {}
    for node in G.nodes():
        neighbors = list(G.neighbors(node))
        if neighbors:
            neighbor = random.choice(neighbors)
            if payoffs[neighbor] > payoffs[node]:
                prob = (payoffs[neighbor] - payoffs[node]) / (max(payoffs.values()) - min(payoffs.values()) + 1e-6)
                if random.random() < prob:
                    new_strategies[node] = G.nodes[neighbor]['strategy']
                else:
                    new_strategies[node] = G.nodes[node]['strategy']
            else:
                new_strategies[node] = G.nodes[node]['strategy']
    for node in G.nodes():
        G.nodes[node]['strategy'] = new_strategies[node]

# Simulation loop
def simulate():
    G = create_lattice()
    cooperation_levels = []

    for t in range(100):  # Simulate for 100 time steps
        payoffs = play_game(G)
        update_strategies(G, payoffs)
        
        num_cooperators = sum(1 for node in G.nodes() if G.nodes[node]['strategy'] == 'C')
        cooperation_level = num_cooperators / NUM_AGENTS
        cooperation_levels.append(cooperation_level)
        
        # Plot the lattice at time steps 1, 50, and 100
        if t in [0, 49, 99]:
            plot_lattice(G, t+1)
    
    return cooperation_levels, G

# Visualization of the lattice with different colors for cooperators and defectors
def plot_lattice(G, timestep):
    # Color cooperators (blue) and defectors (red)
    strategy_map = {'C': 'blue', 'D': 'red'}
    node_colors = [strategy_map[G.nodes[node]['strategy']] for node in G.nodes()]
    
    # Draw the grid with proper node colors
    pos = dict((n, n) for n in G.nodes())  # Use


KeyboardInterrupt: 

Plotting

In [15]:
def plot_results(cooperation_levels, resource_distribution, G):
    # Plot cooperation levels over time
    plt.figure(figsize=(10, 6))
    plt.plot(range(NUM_TIME_STEPS), cooperation_levels, marker='o')
    plt.title('Cooperation Level Over Time')
    plt.xlabel('Time Step')
    plt.ylabel('Cooperation Level')
    plt.grid(True)
    plt.show()

    # Plot resource distribution
    plt.figure(figsize=(10, 6))
    plt.hist(resource_distribution, bins=20, edgecolor='black')
    plt.title('Resource Distribution Among Agents')
    plt.xlabel('Normalized Resource Level')
    plt.ylabel('Number of Agents')
    plt.grid(True)
    plt.show()

    # Visualize network with strategies
    color_map = {'C': 'blue', 'D': 'red'}
    node_colors = [color_map[G.nodes[node]['strategy']] for node in G.nodes()]
    sizes = [G.nodes[node]['resource'] * 1000 for node in G.nodes()]  # Scale sizes for visualization

    plt.figure(figsize=(12, 8))
    nx.draw_networkx(G, node_color=node_colors, node_size=sizes, with_labels=False)
    plt.title('Network Visualization: Blue = Cooperators, Red = Defectors')
    plt.show()

    # Plot Resource Distribution vs Cooperation Level
    strategies = [G.nodes[node]['strategy'] for node in G.nodes()]
    cooperation_flags = [1 if strategy == 'C' else 0 for strategy in strategies]

    plt.figure(figsize=(10, 6))
    plt.scatter(resource_distribution, cooperation_flags, c=cooperation_flags, cmap='coolwarm', alpha=0.7)
    plt.title('Resource Distribution vs Cooperation Level')
    plt.xlabel('Resource Level')
    plt.ylabel('Cooperation (1 = Cooperate, 0 = Defect)')
    plt.grid(True)
    plt.show()

    # Plot Gini Coefficient vs Cooperation Level
    gini = gini_coefficient(resource_distribution)
    plt.figure(figsize=(10, 6))
    plt.bar(['Cooperation Level', 'Gini Coefficient'], [cooperation_levels[-1], gini])
    plt.title('Final Cooperation Level vs Gini Coefficient (Inequality)')
    plt.ylabel('Value')
    plt.show()


Interactive Simulation

In [ ]:
def interactive_simulation(num_agents, cooperation_prob, inequality_level):
    global NUM_AGENTS, INITIAL_COOPERATION_PROB, INEQUALITY_LEVEL
    NUM_AGENTS = num_agents
    INITIAL_COOPERATION_PROB = cooperation_prob
    INEQUALITY_LEVEL = inequality_level

    #Run & plot simulation
    cooperation_levels, resource_distribution, G = simulate()
    plot_results(cooperation_levels, resource_distribution, G)

widgets.interact(interactive_simulation,
                 num_agents=widgets.IntSlider(min=50, max=500, step=50, value=100),
                 cooperation_prob=widgets.FloatSlider(min=0, max=1, step=0.1, value=0.5),
                 inequality_level=widgets.FloatSlider(min=1, max=5, step=0.1, value=2.0))
